# **Scrapping Data Coment Youtube**

In [ ]:
!pip install selenium --quiet


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Scrap

In [ ]:
data=[]

with Chrome(executable_path=r'C:\webdrivers') as driver:
    wait = WebDriverWait(driver,10)
    driver.get("https://youtu.be/UMzBos0eGvc")

    for item in range(10): 
        wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
        time.sleep(5)

    for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):
        data.append(comment.text)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_25440\101441414.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with Chrome(executable_path=r'C:\webdrivers') as driver:


In [ ]:
import pandas as pd   
df = pd.DataFrame(data, columns=['comment'])
df

,comment
0,ID\nSkip navigation\nSign in\nKoalisi Pendukun...
1,Nasib Partai Nasdem di koalisi pemerintahan Jo...
2,Politik itu pilihan dan kepentingan dari parta...
3,Kalaupun benar retak tentunya wajar. Ada orang...
4,negara sdh amburadul kok msh mau d atur dgn ca...
...,...
200,Koalisi Pendukung Jokowi Mulai Retak?\nCNN Ind...
201,Nasib Partai Nasdem di koalisi pemerintahan Jo...
202,
203,


## Prepocessing and cleaning

In [ ]:
import re, string
import numpy as np

# Text Cleaning
def cleaning(text):
    # HTML Tag Removal

    text = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});').sub('', str(text))

    # Case folding
    text = text.lower()

    # Trim text
    text = text.strip()

    # Remove punctuations, karakter spesial, and spasi ganda
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)

    # Number removal
    text = re.sub(r'\[[0-9]*\]', ' ', text)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    # Mengubah text 'nan' dengan whitespace agar nantinya dapat dihapus
    text = re.sub('nan', '', text)

    return text

In [ ]:
!pip install indonlp


     ---------------------------------------- 0.0/121.9 kB ? eta -:--:--
     ------------------- ------------------- 61.4/121.9 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 121.9/121.9 kB 1.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install nltk

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.5 MB 1.7 MB/s eta 0:00:01
     ----- ---------------------------------- 0.2/1.5 MB 2.0 MB/s eta 0:00:01
     --------- ------------------------------ 0.4/1.5 MB 2.9 MB/s eta 0:00:01
     -------------- ------------------------- 0.6/1.5 MB 3.2 MB/s eta 0:00:01
     --------------------- ------------------ 0.8/1.5 MB 3.4 MB/s eta 0:00:01
     --------------------------- ------------ 1.0/1.5 MB 3.8 MB/s eta 0:00:01
     -------------------------------- ------- 1.2/1.5 MB 3.7 MB/s eta 0:00:01
     ------------------------------------- -- 1.4/1.5 MB 3.8 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 3.6 MB/s eta 0:00:00
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
     ---------------------------------------- 0.0/267.9 kB ? eta -:--:--
     ---------- ---------------------------- 71.7/267.9 kB 1.9 MB/s eta 0:00:01
     -----------------


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from indoNLP.preprocessing import pipeline, replace_word_elongation, replace_slang, emoji_to_words, remove_html
pipe = pipeline([replace_word_elongation, replace_slang, emoji_to_words, remove_html])
# steamming
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
df['comment (clean)'] = df['comment'].apply(lambda x: pipe(x))
df['comment (clean)'] = df['comment (clean)'].apply(lambda x: cleaning(x))
# ubah empty string menjadi NaN
df['comment (clean)'] = df['comment (clean)'].replace('', np.nan)

In [ ]:
print(df.isna().sum())

comment            0
comment (clean)    4
dtype: int64


In [ ]:
df.dropna(inplace=True)

## Data clean

In [ ]:
df

,comment,comment (clean)
0,ID\nSkip navigation\nSign in\nKoalisi Pendukun...,id skip navigation sign ini koalisi pendukung ...
1,Nasib Partai Nasdem di koalisi pemerintahan Jo...,nasib partai nasdem di koalisi pemerintahan jo...
2,Politik itu pilihan dan kepentingan dari parta...,politik itu pilihan dan kepentingan dari parta...
3,Kalaupun benar retak tentunya wajar. Ada orang...,kalaupun benar retak tentunya wajar ada orang ...
4,negara sdh amburadul kok msh mau d atur dgn ca...,negara sudah amburadul kok masih mau di atur d...
...,...,...
196,Karena pak jokowi selalu di celah anis basweda...,karena pak jokowi selalu di celah anis basweda...
197,Kasian nasdrun .kurang peka pada sinyal \n,kasihan nasdrun kurang peka pada sinyal
200,Koalisi Pendukung Jokowi Mulai Retak?\nCNN Ind...,koalisi pendukung jokowi mulai retak cn indone...
201,Nasib Partai Nasdem di koalisi pemerintahan Jo...,nasib partai nasdem di koalisi pemerintahan jo...


## Export

In [ ]:
# to csv
df.to_csv('datacomment_cnn_fix.csv', index=False)